In [7]:
import sys
import os
import io

# Add the parent directory to the sys.path
sys.path.append(os.path.join(os.getcwd(), '..'))

In [8]:
import glob
import csv

import numpy as np
import pandas as pd
import seaborn as sns

from scipy.stats import kurtosis, skew, rankdata
from scipy.signal import argrelextrema
from scipy.stats.kde import gaussian_kde

from rcv_learning.rcv_distribution import *
from rcv_learning.rcv_dimensionality import *

/var/folders/g2/25z774tn0599smwks8_v_k200000gp/T/ipykernel_88190/4019494009.py:10: DeprecationWarning: Please use `gaussian_kde` from the `scipy.stats` namespace, the `scipy.stats.kde` namespace is deprecated.
  from scipy.stats.kde import gaussian_kde


In [9]:
def compute_ecdf(data):
    """Compute the empirical cumulative distribution function (ECDF) of data."""
    sorted_data = np.sort(data)
    n = len(data)
    return sorted_data, np.arange(1, n+1) / n

def dip_statistic(data):
    """Compute Hartigan's dip statistic for data."""
    data = np.sort(data)
    n = len(data)
    
    # Compute the empirical CDF
    x, ecdf = compute_ecdf(data)
    
    # Compute the greatest difference on the left and right side for each data point
    U = np.arange(1, n+1) / n
    D = U - np.arange(0, n) / n
    
    # Calculate the difference between the empirical distribution function and the 
    # unimodal distribution function that minimizes that maximum difference.
    low_dip = (ecdf - D).clip(min=0)
    up_dip = (U - ecdf).clip(min=0)
    both_dips = np.column_stack((low_dip, up_dip))
    
    # The dip statistic is the maximum difference over all data points
    dip = np.max(both_dips)
    
    return dip

In [12]:
# Search for CSV files in the directory and its subdirectories
csv_files = glob.glob("../rcv_elections_database/**/*.csv", recursive=True)

# Determine save behavior
save = True

# Dictionary to store all the consistency points across elections
election_consistency_points = {}

# Iterate over the CSV files
for file_path in csv_files:

    # Get the filename only
    filename = file_path.split('/')[-1]  
    print(file_path)

    try:
        csv = file_path
        filename_prefix = "plots/" + filename

        # Check if the plot already exists
        if os.path.isfile(f"{filename_prefix}_hist.png") and False: #! Shouldn't be False
            print("Skipping hist plot: {}".format(file_path))
            continue
            
        # Perform the RCV analysis
        test = perform_rcv_analysis(csv, n_runs=1000)
        mds_1d_coordinates, mds_2d_coordinates, most_common_order, order_frequencies, candidate_names = test

        # Print the normalized distances between candidates and plot the MDS analysis
        normalized_distances = get_distances_normalized(most_common_order, mds_1d_coordinates, candidate_names)
        print("Normalized distances:", normalized_distances)
        plot_rcv_analysis(mds_1d_coordinates, mds_2d_coordinates, most_common_order, order_frequencies, candidate_names, save=save, filename=filename_prefix)

        # Get the consistency points for the bimodality analysis
        points = get_consistency_points(csv)
        print("Consistency points:", points)
        
        # Store the consistency points for later normalization and plotting
        election_consistency_points[filename] = points

        # Create a list of data points
        data_points = []
        for key, value in points.items():
            data_points.extend([key] * value)

        # Convert to numpy array
        data_points = np.array(data_points)

        # Calculate skewness and kurtosis
        g = skew(data_points)
        k = kurtosis(data_points)

        # Calculate KDE without plotting
        density = gaussian_kde(data_points)
        x_vals = np.linspace(min(data_points), max(data_points), 1000)
        y_vals = density(x_vals)

        # Identify local maxima in the KDE
        maxima_indices = argrelextrema(y_vals, np.greater)

        # Get the x-values of the maxima
        modes = x_vals[maxima_indices]

        # Check if there are at least two modes
        if len(modes) >= 2:
            mode1, mode2 = modes[:2]

            # Calculate the amplitudes of the two modes
            amp1 = density(mode1)[0]
            amp2 = density(mode2)[0]

            # Calculate the amplitude ratio with higher amplitude as denominator
            if amp1 > amp2:
                amplitude_ratio = amp2 / amp1
            else:
                amplitude_ratio = amp1 / amp2

            # Calculate x-axis distance between the two modes
            mode_distance = abs(mode2 - mode1)

        else:
            mode1, mode2 = None, None
            amplitude_ratio = None
            mode_distance = None

        try:
            # Split the data into two groups based on proximity to the modes
            data_group1 = [point for point in data_points if abs(point - mode1) < abs(point - mode2)]
            data_group2 = [point for point in data_points if abs(point - mode1) > abs(point - mode2)]

            # Display mean and variance for each group
            mu1, sigma1_sq = np.mean(data_group1), np.var(data_group1)
            mu2, sigma2_sq = np.mean(data_group2), np.var(data_group2)

            # Calculate Ashman's D statistic (D > 2)
            ashmans_D = abs(mu1 - mu2) / np.sqrt((sigma1_sq + sigma2_sq) / 2)

        except:
            mode_distance = 404
            amplitude_ratio = 404
            ashmans_D = 404

        # Calculate Sarle's bimodality coefficient b (b > 5/9)
        n = len(data_points)
        sarle = (g**2 + 1) / (k + 3 * (n-1)**2 / ((n-2) * (n-3)))

        # Calculate Hartigan's dip statistic (calculate p value)
        hartigan_dip = dip_statistic(data_points)

        # Prepare data for histogram
        data_list = [x for x, count in points.items() for _ in range(count)]
        normalized_points = []
        normalized_names = []
        for name in normalized_distances:
            normalized_names.append(name)
            normalized_points.append(normalized_distances[name])

        # Plot histogram
        plt.figure(figsize=(10, 6))
        plt.hist(data_list, bins=50, density=True, alpha=0.7)
        plt.title('Histogram of Data')
        plt.xticks(normalized_points, normalized_names, rotation=45)
        plt.xlabel('Value')
        plt.ylabel('Density')
        plt.grid(True)

        # Display measures in a text box
        text_str = f"Skewness: {g:.2f}\nKurtosis: {k:.2f}\nMode Distance: {mode_distance:.2f}"
        plt.text(0.05, 0.95, text_str, transform=plt.gca().transAxes, fontsize=10, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))

        if save:
            plt.savefig(f"{filename_prefix}_hist.png", bbox_inches='tight')
            plt.close()
        else:
            plt.show()

        # Plot kernal density estimation
        plt.figure(figsize=(10, 6))
        sns.kdeplot(data_list, fill=True)
        plt.title('Kernel Density Estimation of Data')
        plt.xticks(normalized_points, normalized_names, rotation=45)
        plt.xlabel('Value')
        plt.ylabel('Density')
        plt.grid(True)

        # Display bimodality measures in a text box
        text_str = f"Ashman's D: {ashmans_D:.10f}\nSarle's Coefficient: {sarle:.10f}\nHartigan's Dip: {hartigan_dip:.10f}\nAmplitude Ratio: {amplitude_ratio:.10f}"
        plt.text(0.05, 0.95, text_str, transform=plt.gca().transAxes, fontsize=10, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))

        if save:
            plt.savefig(f"{filename_prefix}_kde.png", bbox_inches='tight')
            plt.close()
        else:
            plt.show()

    except Exception as e:
        print("Error in file: {}".format(file_path))
        print(e)

../rcv_elections_database/CandidateDetails.csv
Error in file: ../rcv_elections_database/CandidateDetails.csv
zero-size array to reduction operation minimum which has no identity
../rcv_elections_database/SingleWinnerRCV.csv
Error in file: ../rcv_elections_database/SingleWinnerRCV.csv
zero-size array to reduction operation minimum which has no identity
../rcv_elections_database/MatchedElections.csv
Error in file: ../rcv_elections_database/MatchedElections.csv
zero-size array to reduction operation minimum which has no identity
../rcv_elections_database/SequentialRCV.csv
Error in file: ../rcv_elections_database/SequentialRCV.csv
zero-size array to reduction operation minimum which has no identity
../rcv_elections_database/ProportionalRCV.csv
Error in file: ../rcv_elections_database/ProportionalRCV.csv
zero-size array to reduction operation minimum which has no identity
../rcv_elections_database/proportional/Minneapolis_11072017_BoardofEstimateandTaxation.csv


/Users/es5891/Documents/GitHub/bugs-in-democracy/rcv_exploration/../rcv_learning/rcv_dimensionality.py:177: RuntimeWarning: invalid value encountered in divide
  frequencies = counts / mentioned_together
/Users/es5891/Documents/GitHub/bugs-in-democracy/rcv_exploration/../rcv_learning/rcv_dimensionality.py:188: RuntimeWarning: divide by zero encountered in divide
  distance = 1 / np.sqrt(freq_upper_triangle)


Normalized distances: {'Carol Becker': 0.0, 'David B. Wheeler': 0.9489056136205803, 'John Edwards': 2.0}


/Users/es5891/Documents/GitHub/bugs-in-democracy/rcv_exploration/../rcv_learning/rcv_dimensionality.py:188: RuntimeWarning: divide by zero encountered in divide
  distance = 1 / np.sqrt(freq_upper_triangle)


Consistency points: {1.0510943863794198: 7191, 2.0: 17436, 1.2883207897845648: 11544, 0.26277359659485494: 648, 0.5: 62, 0.3813867982974275: 147, 0.0: 598, 1.1569339914871373: 37}
../rcv_elections_database/proportional/Minneapolis_11072017_ParkBoardAtLarge.csv


...

Consistency points: {0.17944882560752065: 293, 0.5101675553378401: 506, 0.4571005160755658: 343, 0.9540235243053404: 18, 0.15591031407965356: 182, 0.0968532586108391: 99, 0.3874130344433564: 416, 0.5526041684367196: 20, 0.0: 312, 0.859869478193872: 146, 0.214967369548468: 75, 0.5055271453809853: 173, 0.5996811914924537: 21, 0.2620443926042022: 19, 0.8707531541838162: 39, 0.8465398395311065: 15, 1.2364856626397456: 23, 4.0: 12, 0.3091214156599364: 14, 0.894966468836526: 3, 1.9273642469798091: 2, 1.2905597758325174: 3, 0.9688822329902577: 1, 0.596527378380348: 2, 1.644902108645404: 2}
../rcv_elections_database/single/SanLeandro_11032020_MemberCityCouncilDist2SanLeandroRCV.csv


/Users/es5891/Documents/GitHub/bugs-in-democracy/rcv_exploration/../rcv_learning/rcv_dimensionality.py:188: RuntimeWarning: divide by zero encountered in divide
  distance = 1 / np.sqrt(freq_upper_triangle)


Normalized distances: {'ED HERNANDEZ': 0.0, 'BRYAN AZEVEDO': 1.0}


/Users/es5891/Documents/GitHub/bugs-in-democracy/rcv_exploration/../rcv_learning/rcv_dimensionality.py:188: RuntimeWarning: divide by zero encountered in divide
  distance = 1 / np.sqrt(freq_upper_triangle)


Consistency points: {0.0: 5560, 1.0: 6439, 0.25: 10302}


In [13]:
import pickle

# Specify the filename for the pickle file
pickle_filename = "election_consistency_points.pkl"

# Open the file in write-binary mode and dump the dictionary
with open(pickle_filename, 'wb') as f:
    pickle.dump(election_consistency_points, f)

In [ ]:
# Open the pickle file in read-binary mode and load the dictionary
with open(pickle_filename, 'rb') as f:
    election_consistency_points = pickle.load(f)

In [ ]:
def average_and_plot(points_list):
    
    # Normalize the dictionaries
    normalized_points_list = []
    for points in points_list:
        total = sum(points.values())
        normalized = {k: v / total for k, v in points.items()}
        normalized_points_list.append(normalized)

    # Average across the normalized dictionaries
    averaged_points = {}
    for normalized_points in normalized_points_list:
        for k, v in normalized_points.items():
            if k in averaged_points:
                averaged_points[k] += v
            else:
                averaged_points[k] = v

    num_dicts = len(normalized_points_list)
    averaged_points = {k: v / num_dicts for k, v in averaged_points.items()}

    # Extract data for histogram and KDE
    data_list = [x for x, count in averaged_points.items() for _ in range(int(count * 100000))]  # Scale the values for visualization

    # Plot histogram
    plt.figure(figsize=(10, 6))
    plt.hist(data_list, bins=50, density=True, alpha=0.7)
    plt.title('Averaged Histogram of Data')
    plt.xlabel('Value')
    plt.ylabel('Density')
    plt.grid(True)
    plt.show()

    # Plot kernal density estimation
    plt.figure(figsize=(10, 6))
    sns.kdeplot(data_list, fill=True)
    plt.title('Averaged Kernel Density Estimation of Data')
    plt.xlabel('Value')
    plt.ylabel('Density')
    plt.grid(True)
    plt.show()

In [ ]:
def filter_elections(level, partisan_value):
    """Filter the elections based on the level and partisan value."""
    filtered_elections = election_df[(election_df['level'] == level) & (election_df['partisan'] == partisan_value)]['filename'].tolist()
    return [election_consistency_points[election] for election in filtered_elections if election in election_consistency_points]

# Filter and average for STATE and FEDERAL level elections with YES in partisan
state_federal_points = filter_elections('FEDERAL', 'YES') + filter_elections('STATE', 'YES')
print("Averaging for STATE and FEDERAL level elections with YES in partisan:")
average_and_plot(state_federal_points)

# Filter and average for LOCAL elections with NO in partisan
local_points = filter_elections('LOCAL', 'NO')
print("Averaging for LOCAL elections with NO in partisan:")
average_and_plot(local_points)